In [1]:
from summarization_algorithm import *
from preprocessing_algorithms import *

c:\Users\Zeyad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import pandas as pd
data = pd.read_csv('DUC/Duc_first_reference/main_dataset/Duc_dataset_first_ref_summary.csv')
text = data.iloc[1,0]
text

"\r\nHonduras braced for potential catastrophe Tuesday as Hurricane Mitch \r\nroared through the northwest Caribbean, churning up high waves and \r\nintense rain that sent coastal residents scurrying for safer ground. \r\nPresident Carlos Flores Facusse declared a state of maximum alert \r\nand the Honduran military sent planes to pluck residents from their \r\nhomes on islands near the coast. At 0900 GMT Tuesday, Mitch was 95 \r\nmiles (152 kilometers) north of Honduras, near the Swan Islands. With \r\nwinds near 180 mph (289 kph), and even higher gusts, it was a Category \r\n5 monster _ the highest, most dangerous rating for a storm. The 350-mile \r\n(560-kilometer) wide hurricane was moving west at 8 mph (12 kph). \r\n``Mitch is closing in,'' said Monterrey Cardenas, mayor of Utila, \r\nan island 20 miles (32 kilometers) off the Honduran coast. ``And God \r\nhelp us.'' Mitch posed no immediate threat to the United States, forecasters \r\nsaid, but was expected to remain in the north

In [4]:
# # define the input text
# #
# # load the GPT-2 tokenizer and model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2Model.from_pretrained('gpt2')
# # tokenize the input text into sentences
# sentences = text.split('. ')

# # compute the embedding of each sentence using GPT-2
# embeddings = []
# for sentence in sentences:
#     input_ids = tokenizer.encode(sentence, return_tensors='pt')
#     with torch.no_grad():
#         output = model(input_ids)[0][:, -1, :]
#         embeddings.append(output)
# # compute the similarity between each sentence embedding and the embedding of the entire text
# text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
# similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0), text_embedding)

# # rank the sentences based on their similarity score
# sentence_scores = list(zip(sentences, similarity_scores))
# sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

# # select the top N sentences to form the summary
# N = 5
# summary = [s[0] for s in sentence_scores[:N]]

In [5]:
def gpt2_extractive_summarization(filtered_sentences,text, num_sentences):
    # load the GPT-2 tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')

    # tokenize the input text into sentences
    sentences = text.split('. ')
    print(sentences)

    # compute the embedding of each sentence using GPT-2
    embeddings = []
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        with torch.no_grad():
            output = model(input_ids)[0][:, -1, :]
            embeddings.append(output)

            

    # compute the similarity between each sentence embedding and the embedding of the entire text
    text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
    return embeddings,text_embedding
    similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0).numpy(), text_embedding.numpy())


    # rank the sentences based on their similarity score
    sentence_scores = list(zip(text, similarity_scores))
    sentence_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)

    # select the top N sentences to form the summary
    summary = [s[0] for s in sentence_scores[:num_sentences]]
    return '. '.join(summary)


In [20]:
def gpt2_extractive_summarization(sentences, filtered_sentences, size):
    # load the GPT-2 tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')

    # compute the embedding of each sentence using GPT-2
    embeddings = []
    for sentence in filtered_sentences:
        input_ids = tokenizer.encode(sentence, return_tensors='pt')
        with torch.no_grad():
            output = model(input_ids)[0][:, -1, :]
            embeddings.append(output)

    # compute the similarity between each sentence embedding and the embedding of the entire text
    text_embedding = torch.mean(torch.cat(embeddings, dim=0), dim=0, keepdim=True)
    similarity_scores = cosine_similarity(torch.cat(embeddings, dim=0).numpy(), text_embedding.numpy())

    # rank the sentences based on their similarity score
    sentence_scores = list(zip(sentences, similarity_scores))
    top_scores = sorted(sentence_scores, key=lambda x: x[1], reverse=True)[:size]

    # select the top N sentences to form the summary
    summary = [s[0] for s in sentence_scores if s in top_scores]
    scores = [s[1] for s in sentence_scores if s in top_scores]
    return summary


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Original sentences
original_sentences = ["The quick brown fox jumps over the lazy dog.",
                       "This is a sentence.",
                       "Another sentence here.",
                       "This is a short sentence.",
                       "The dog is lazy."]

# Preprocess the sentences
filtered_sentences = []
for sentence in original_sentences:
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence.lower())
    
    # Remove stopwords and lemmatize the remaining words
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    
    # Convert the filtered words back to a sentence and add it to the list of filtered sentences
    filtered_sentence = ' '.join(filtered_words)
    filtered_sentences.append(filtered_sentence)

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Zeyad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
original_sentences

['The quick brown fox jumps over the lazy dog.',
 'This is a sentence.',
 'Another sentence here.',
 'This is a short sentence.',
 'The dog is lazy.']

In [11]:
gpt2_extractive_summarization(filtered_sentences,original_sentences,5)

In [12]:
from preprocessing_algorithms import *

In [23]:
article_str_sentences, processed_sentences = preprocessing_text_with_spacy(text, False, False)

In [14]:
import pandas as pd
df = pd.read_csv("DUC\Duc_first_reference\main_dataset\Duc_dataset_first_ref_summary.csv")

In [15]:
sentences, processed_sentences = process_one_column_df(df["Original Article"])

100%|██████████| 49/49 [02:01<00:00,  2.48s/it]


In [24]:
gpt2_extractive_summarization(processed_sentences,article_str_sentences,5)

TypeError: cosine_similarity() missing 2 required positional arguments: 'i' and 'j'

In [17]:
from summarization_algorithm import *

In [21]:
summarize_with(processed_sentences, sentences,gpt2_extractive_summarization, 5)

  0%|          | 0/49 [00:16<?, ?it/s]


TypeError: cosine_similarity() missing 2 required positional arguments: 'i' and 'j'